In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sb
from tensorflow.keras.preprocessing.image import load_img, img_to_array

****datamızdan bazı verileri gösterdik****

In [63]:
data=pd.read_csv(r"../input/hardfakevsrealfaces/data.csv")
data.head()

***datamızda bulunan resimlerin boyutlarını 128 128 olarak hepsini standart hala getirdik***

***sonrasında ne kadar elimizde data var bunu gösterdik***

In [64]:
height, width=128, 128
X=np.empty((data.shape[0], height, width, 3))
for i in range(data.shape[0]):
    img=load_img(r"../input/hardfakevsrealfaces/{}/{}.jpg".format(data.loc[i, 'label'], \
                  data.loc[i, 'images_id']), target_size=(height, width))
    X[i]=img_to_array(img)
X.shape

*labelarımızı değiştirdik [0 1]  , [1 0]*

In [65]:
def changeLabels(x):
    return labels[x]

labels=data.label.unique()
labels={labels[i]:i for i in range(labels.size)}
y=data.label.apply(changeLabels)
y[:5]

In [66]:
from tensorflow.keras.utils import to_categorical

In [67]:
y=to_categorical(y, len(labels))
y=y.astype(int)
y[:5]

***datamızı train test olarak ayırmak için sklearn kütüphanesinden train_test_split metodunu çağırdık***

In [68]:
from sklearn.model_selection import train_test_split

***data setimizi %80 %20 oranında train test olarak ayırdık***

In [69]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=8)
X_train.shape, y_train.shape

In [87]:
X_test.shape , y_test.shape

# Oluşturmuş olduğumuz datasetini görselleştiriyoruz

In [70]:
resimler = []
for i in range(data.shape[0]):
    img=load_img(r"../input/hardfakevsrealfaces/{}/{}.jpg".format(data.loc[i, 'label'], \
                  data.loc[i, 'images_id']), target_size=(height, width))
    resimler.append(img)

In [71]:
plot.imshow(resimler[14])

In [72]:
plot.imshow(resimler[15])

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

In [74]:
model=Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(height, width, 3)))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))


model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.45))
model.add(Dense(2, activation='softmax'))

***epoch olarak 15 den sonra ezberlemeye kaydığını düşündüğümden 12 de epoch sayımızı belirledik , 
train datasetinden de %20 sini validation olarak ayarladık***

In [75]:
epochs=15
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
stats=model.fit(X_train, y_train, epochs=epochs, validation_split=0.2)
model.summary()

***Oluşturduğumuz Sinir Ağını 3 boyutlu model ile gösteriyoruz***

In [76]:
!pip install git+https://github.com/paulgavrikov/visualkeras

In [77]:
# Visualizing our model (Hidden Input)
import visualkeras
visualkeras.layered_view(model, scale_xy=10, legend=True)

In [78]:
stats_df=pd.DataFrame(stats.history)
stats_df['epoch']=list(range(1, epochs+1))
plot.figure(figsize=(10, 8))
sb.lineplot(y='loss', x='epoch', data=stats_df, color='deeppink', linewidth=2.5, label="Training loss")
sb.lineplot(y='val_loss', x='epoch', data=stats_df, color='darkturquoise', linewidth=2.5, label="Validation loss")
plot.grid()
plot.legend()
plot.title("Training and validation loss")

In [79]:
stats_df=pd.DataFrame(stats.history)
stats_df['epoch']=list(range(1, epochs+1))
plot.figure(figsize=(10, 8))
sb.lineplot(y='accuracy', x='epoch', data=stats_df, color='deeppink', linewidth=2.5, label="Training accuracy")
sb.lineplot(y='val_accuracy', x='epoch', data=stats_df, color='darkturquoise', linewidth=2.5, \
            label="Validation accuracy")
plot.grid()
plot.legend()
plot.title("Training and validation accuracy")

In [80]:
print('Accuracy:')
_, accuracy=model.evaluate(X_test, y_test)

# **Confusion Matrix**

**Confusion Matrix sayesinde hangi test verilerin hangi sınıflara tahmin edildiğini gösteriyor**

In [81]:
class_names = ['Real', 'Fake']

In [82]:
import seaborn as sns
import tensorflow as tf

In [83]:
# Predict the values from the validation dataset
y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

In [84]:
plot.figure(figsize=(12, 9))
c = sns.heatmap(confusion_mtx, annot=True, fmt='g')
c.set(xticklabels=class_names, yticklabels=class_names)